In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1187,  0.1882, -0.0438,  0.0997, -0.1191, -0.0309,  0.0227, -0.0279,
          0.1391, -0.1949],
        [-0.0601,  0.0748, -0.0941,  0.0103, -0.0605, -0.0093,  0.0418,  0.0697,
          0.0921, -0.0375]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.2606, -0.0977,  0.0510,  0.1490, -0.1600,  0.0764,  0.0704,  0.0566,
         -0.0845, -0.0467],
        [ 0.2347, -0.2219, -0.0648,  0.1765,  0.0125,  0.1111,  0.0093,  0.0489,
         -0.1095,  0.0510]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0999,  0.0850,  0.1282,  0.0153,  0.1057,  0.0914,  0.1790, -0.0916,
          0.0982, -0.1603],
        [-0.2534, -0.0338,  0.1217, -0.0309,  0.1653,  0.1154,  0.2047, -0.0088,
          0.0932, -0.0959]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2605, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1271, grad_fn=<SumBackward0>)